## Предсказания победителя в онлайн-игре

### Этап 1. Градиентный бустинг "в лоб"

In [9]:
import pandas as pd

df = pd.read_csv('features.csv', index_col='match_id')
df.head(7)

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63
5,1430284186,1,11,5,1961,1461,19,0,1,6,...,4,4,0,-43.0,1453,0,512,2038,0,63
8,1430293701,1,8,3,967,1136,7,1,0,8,...,6,3,0,10.0,1968,0,1536,1983,12,63


In [10]:
print(df.shape)

(97230, 108)


#### 1.1.  В обучающей выборке удаляем столбцы с признаками, использующими итоги матча

In [11]:
X = df.drop(labels=['duration', 'radiant_win', 'tower_status_radiant', 'tower_status_dire', 
                    'barracks_status_radiant', 'barracks_status_dire'], axis=1)
X.shape

(97230, 102)

#### 1.2. Опрелеляем столбцы, имеющие значения NaN

In [12]:
X.columns[X.isnull().sum() > 0]

Index(['first_blood_time', 'first_blood_team', 'first_blood_player1',
       'first_blood_player2', 'radiant_bottle_time', 'radiant_courier_time',
       'radiant_flying_courier_time', 'radiant_first_ward_time',
       'dire_bottle_time', 'dire_courier_time', 'dire_flying_courier_time',
       'dire_first_ward_time'],
      dtype='object')

#### 1.3. Заменяем пропуски в данных на значение 360

In [13]:
X = X.fillna(360)

#### 1.4. Столбец radiant_win опрелеляем в качестве целевого

In [14]:
Y = df['radiant_win']
Y.shape

(97230,)

#### 1.5. Использование градиентного бустинга

In [15]:
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingClassifier

import time
import datetime

# преобразуем обучающие выборки и целевой столбец в массивы NumPy
X = np.array(X)
y = np.array(Y)

# определим модель
clf = GradientBoostingClassifier(learning_rate=0.1, random_state=999)

# оценим качество модели на кросс-валидации с 5-ю блоками при количестве деревьев 10, 20, 30, 40, 50, 100, 150
n_trees = [10, 20, 30, 40, 50, 100, 150]

cv = KFold(n_splits=5, shuffle=True, random_state=999)

# старт расчета
start_time = datetime.datetime.now()

for n_tree in n_trees:
    lst=np.zeros(cv.n_splits)
    clf.n_estimators = n_tree
    j=0
    for train_indexes, test_indexes in cv.split(X):
        clf.fit(X[train_indexes], y[train_indexes])
        pred = clf.predict_proba(X[test_indexes])
        lst[j] = roc_auc_score(y[test_indexes], pred[:,1])
        j+=1
        
    print('n_tree = ' + str(clf.n_estimators), '   auc_roc = ' + str(lst.mean()))
    
print('Time elapsed:', datetime.datetime.now() - start_time)

n_tree = 10    auc_roc = 0.667032413488
n_tree = 20    auc_roc = 0.682769823568
n_tree = 30    auc_roc = 0.689392310643
n_tree = 40    auc_roc = 0.694472005164
n_tree = 50    auc_roc = 0.697800725516
n_tree = 100    auc_roc = 0.706810886584
n_tree = 150    auc_roc = 0.711196956907
Time elapsed: 0:12:49.527089


## Отчет по этапу 1

1. В 12-и столбцах исходных данных имеются пропуски: first_blood_time, first_blood_team, first_blood_player1, first_blood_player2, radiant_bottle_time, radiant_courier_time, radiant_flying_courier_time, radiant_first_ward_time, dire_bottle_time, dire_courier_time, dire_flying_courier_time, dire_first_ward_time.
   
   Все эти признаки связаны с временем наступления того или иного события, и пропуски данных в этих случаях отвечают ситуации, когда событие не возникло в первые 300 секунд игры. В этих случаях значения NaN можно заменить на число большее 300, напр., на 360 (что более правильно, чем заменять нулем - при этом, как показывают расчеты, получаем небольшой выигрыш в качестве модели).
   
2. Целевые данные находятся в столбце **radiant_win**, который определяет результат матча (1 - побед Radiant или 0 в противном случае).

3. Кросс-валидация для градиентного бустинга с 30 деревьями на ПК с процессором Intel i7-6700K@4.0GHz и встроенной графикой заняла 58.6 секунд. Качество модели определялось по площади ROC-кривой и составило в этом случае 0.6794.

4. Использование более 30 деревьев в градиентном бустинге существенно не добавляет качества моделм, однако заметно увеличивает время обработки. Так при 50 деревьях время, затраченное на обучение, составило 97 секунд при качестве модели 0.6978, а при 150 деревьях - 0.7112 и 286 секунд соответственно. Повысить скорость обучения можно уменьшив количество рассматриваемых признаков, либо проводить обучение на подмножестве объектов исходной выборки.

### Этап 2. Логистическая регрессия

In [17]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

import time
import datetime

df = pd.read_csv('features.csv', index_col='match_id')

# целевой столбец в виде массива NumPy
y = np.array(df['radiant_win'])

# удаляем признаки, зависящие от итогов матча
df = df.drop(labels=['duration', 'radiant_win', 'tower_status_radiant', 'tower_status_dire', 
                     'barracks_status_radiant', 'barracks_status_dire'], axis=1)

# формируем "мешок слов" из категориальных признаков
X_pick = np.zeros((df.shape[0], 112))
for i, match_id in enumerate(df.index):
    for p in range(5):
        X_pick[i, df.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, df.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1
X_pick = np.delete(X_pick,[23,106,107,110], axis=1)  # героев с идентификаторами 24,107,108,111 нет

# удаляем преобразованные категориальные признаки
df = df.drop(labels=['lobby_type','r1_hero','r2_hero','r3_hero','r4_hero','r5_hero',
                                  'd1_hero','d2_hero','d3_hero','d4_hero','d5_hero'], axis=1)

# получаем из dataFrame массив "объекты-признаки", заменяя пропущенные значения на 360 
X = np.array(df.fillna(360))

# масштабируем признаки
scaler = StandardScaler()
X = scaler.fit_transform(X)

# добавляем признаки "мешка слов"
X = np.hstack((X, X_pick))
print(X.shape)

# определим модель
clf = LogisticRegression(solver='liblinear', max_iter=500)  # 'lbfgs'

# набор коэффициентов регуляризации
Cs = [0.05, 1.0, 2.0, 3.0, 4.0, 5.0, 7.0]

cv = KFold(n_splits=5, shuffle=True, random_state=999)

# старт расчета
start_time = datetime.datetime.now()

for c in Cs:
    clf.C = c
    lst=np.zeros(cv.n_splits)
    j=0
    for train_indexes, test_indexes in cv.split(X):
        clf.fit(X[train_indexes], y[train_indexes])
        pred = clf.predict_proba(X[test_indexes])
        lst[j] = roc_auc_score(y[test_indexes], pred[:,1])
        j+=1
        
    print('C = ' + str(clf.C), '   auc_roc = ' + str(lst.mean()))
    
print('Duration: ', datetime.datetime.now() - start_time)


(97230, 199)
C = 0.05    auc_roc = 0.752305056997
C = 1.0    auc_roc = 0.752323479063
C = 2.0    auc_roc = 0.752326227723
C = 3.0    auc_roc = 0.752325983123
C = 4.0    auc_roc = 0.75232687674
C = 5.0    auc_roc = 0.752326817583
C = 7.0    auc_roc = 0.752327291856
Duration:  0:03:41.317139


### Прогноз лучшей модели на тестовой выборке

In [18]:
# обучение лучшей модели на всей исходной выборке
cv = KFold(n_splits=5, shuffle=True, random_state=999)
clf = LogisticRegression(C=4.0, solver='liblinear', max_iter=500)
clf.fit(X, y)

# получение и подготовка тестовой выборки
df_test = pd.read_csv('features_test.csv', index_col='match_id')

# формируем "мешок слов" из категориальных признаков тестовой выборки
X_pick = np.zeros((df_test.shape[0], 112))
for i, match_id in enumerate(df_test.index):
    for p in range(5):
        X_pick[i, df_test.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, df_test.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1
X_pick = np.delete(X_pick,[23,106,107,110], axis=1)  # героев с идентификаторами 24,107,108,111 нет

# удаляем преобразованные категориальные признаки
df_test = df_test.drop(labels=['lobby_type','r1_hero','r2_hero','r3_hero','r4_hero','r5_hero',
                               'd1_hero','d2_hero','d3_hero','d4_hero','d5_hero'], axis=1)

# получаем из dataFrame массив "объекты-признаки", заменяя пропущенные значения на 360 
X_test = np.array(df_test.fillna(360))

# масштабируем признаки
scaler = StandardScaler()
X_test = scaler.fit_transform(X_test)

# добавляем признаки "мешка слов"
X_test = np.hstack((X_test, X_pick))
print(X_test.shape)
    
# максимальный и минимальный прогноз для обученной модели на тестовой выборке
y_test = clf.predict_proba(X_test)
print('min probability = ', y_test.min())
print('max probability = ', y_test.max())

(17177, 199)
min probability =  0.003285399786
max probability =  0.996714600214


## Отчет по этапу 2

1. Качество логистической регрессии при анализе всех исходных признаков для коэффициента регуляризации C = 0.1 составило 0.7167. Оно оказалось даже выше качества градиентного бустинга. Видимо, наличие большого количества признаков в данной задаче смещает выбор модели в пользу логистической регрессии. При этом логистическая регрессия работает примерно в 3 разa Быстрее, чем аналогичная модель градиентного бустинга.

2. Качество логистической регрессии при анализе непреобразованных категориальных признаков и модели, когда эти признаки исключены, практически одинаково. Это связано с тем, что категориальные признаки, кодирующие героев в виде целых чисел, слабо коррелируют с целевой переменной, т.к. в таком виде признаки несравнимы между собой. Преобразование категориальных признаков с использованием "мешка слов" заметно улучшает качество модели.

3. Всего в матчах используется 108 различных идентификаторов героев, закодированных целыми числами в диапазоне от 1 до 112 (идентификаторы 24, 107, 108 и 111 не встречаются в исходных данных).

4. Корректный учет такого важного признака как тип героя заметно и положительно сказывается на качестве модели. При замене пропущенных в исходных данных значений нулем, подбор лучшего параметра регуляризации на модели с "мешком слов" позволяет достичь качества модели 0.7519 при параметре регуляризации C = 0.05. При замене пропущенных значений на 360 качество модели становится чуть выше - 0.7523 и слабо зависит от коэффициента регуляризации. Для лучшей модели принят коэффициент С = 4.0.

5. Минимальное и максимальное значение прогноза на тестовой выборке (минимальная и максимальная вероятность выигрыша команды Radiant) для лучшего из алгоритмов прогнозирования (логистическая регрессия, С=4.0, замена пропущенных значений на 360) оказалось равным 0.0033 и 0.997 соответственно.